In [ ]:
import sys

sys.path.append('../../')

In [ ]:

from matplotlib import rcParams
from tqdm import tqdm
import torch
from omegaconf import OmegaConf

% matplotlib inline
rcParams['figure.figsize'] = (10, 15)

In [ ]:
from torchok.constructor.config_structure import ConfigParams
from torchok.constructor import TASKS

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Path to yaml
config_path = 'examples/configs/representation_arcface_sop.yaml'
# Load yaml
config = OmegaConf.load(config_path)
OmegaConf.resolve(config)
schema = OmegaConf.structured(ConfigParams)
config = OmegaConf.merge(schema, config)

# Path to checkpoint
checkpoint = torch.load('..logs/sop_arcface/sop_arcface/2022-06-28_19-33-57/16-15810.ckpt')

# Create task
model = TASKS.get(config.task.name)(config)
model = model.to(device)
model.load_state_dict(checkpoint['state_dict'])
model.eval();

In [ ]:
# Create vectors on validation dataset
dataloader = model.val_dataloader()[0]

targets = []
vectors = []
for batch in tqdm(dataloader):
    targets.append(batch['target'])
    batch['image'] = batch['image'].to(device)
    vectors.append(model.forward_with_gt(batch)['embeddings'].detach().cpu())

In [ ]:
# Save traced model
traced = torch.jit.trace(model.forward, torch.rand(4, 3, 224, 224, device=device))
traced.save('sop_resnet50_arcface.pt')